In [1]:
products = [
    {
        "product_name": "떡케익5호",
        "quantity": 1,
        "price": 54000
    },
    {
        "product_name": "무지개 백설기 케익",
        "quantity": 1,
        "price": 51500
    },
    {
        "product_name": "미니 백설기",
        "quantity": 35,
        "price": 31500
    },
    {
        "product_name": "개별 모듬팩",
        "quantity": 1,
        "price": 13500
    }
]

In [12]:
def fetch_products(dict):
    return products

In [13]:
from pydantic import BaseModel, Field, conint, condecimal
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

class OrderItem(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    quantity: conint(gt=0) = Field(..., description="Quantity of the product")
    price: condecimal(max_digits=10, decimal_places=2) = Field(..., description="Price of the product")

class CreateOrderData(BaseModel):
    user_id: int = Field(..., description="ID of the user creating the order")
    items: List[OrderItem] = Field(..., description="List of items to order")

create_order_parser = PydanticOutputParser(pydantic_object=CreateOrderData)

In [14]:
from langchain_core.prompts import PromptTemplate

extract_order_args_prompt = PromptTemplate(
    template="""
    You are a robot designed to extract necessary parameters for processing each order request.
    Only include the extracted parameters in your response.

    Product Information:
    {products}

    User ID: {user_id}

    User Input: {input}

    Provide only the extracted parameters in the following format:
    {format_instructions}
    """,
    input_variables=["user_id", "input", "products"],
    partial_variables={"format_instructions": [create_order_parser.get_format_instructions()]},
)

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [25]:
extract_order_args_chain = extract_order_args_prompt | model | create_order_parser
extract_order_args_chain

PromptTemplate(input_variables=['input', 'products', 'user_id'], partial_variables={'format_instructions': ['The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"OrderItem": {"properties": {"product_name": {"description": "Name of the product", "title": "Product Name", "type": "string"}, "quantity": {"description": "Quantity of the product", "exclusiveMinimum": 0, "title": "Quantity", "type": "integer"}, "price": {"anyOf": [{"type": "number"}, {"type": "string"}], "description": "Price of the product", "title": "Price"}}, "required": ["product_name", "quantity", "price"], "tit

In [26]:
from langchain_core.runnables import RunnablePassthrough


extrac_order_chain = RunnablePassthrough.assign(products=fetch_products) | extract_order_args_chain
extrac_order_chain

RunnableAssign(mapper={
  products: RunnableLambda(fetch_products)
})
| PromptTemplate(input_variables=['input', 'products', 'user_id'], partial_variables={'format_instructions': ['The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"OrderItem": {"properties": {"product_name": {"description": "Name of the product", "title": "Product Name", "type": "string"}, "quantity": {"description": "Quantity of the product", "exclusiveMinimum": 0, "title": "Quantity", "type": "integer"}, "price": {"anyOf": [{"type": "number"}, {"type": "string"}], "description": "Price of the product", "ti

In [23]:
RunnablePassthrough.assign(products=fetch_products).invoke({
    "input": "미니 백설기 3개 주문할게요",
    "user_id": 3
})

{'input': '미니 백설기 3개 주문할게요',
 'user_id': 3,
 'products': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000},
  {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500},
  {'product_name': '미니 백설기', 'quantity': 35, 'price': 31500},
  {'product_name': '개별 모듬팩', 'quantity': 1, 'price': 13500}]}

In [27]:
response = extrac_order_chain.invoke({
    "input": "미니 백설기 3개 주문할게요",
    "user_id": 3
})
response

CreateOrderData(user_id=3, items=[OrderItem(product_name='미니 백설기', quantity=3, price=Decimal('31500'))])

In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.
            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),

    ]
)

In [35]:
classify_message_chain = message_type_prompt | model

In [36]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

def add_memory(runnable, session_id):
    runnable_with_memory = RunnableWithMessageHistory(
        runnable,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        )
    memory_by_session = RunnableLambda(
        lambda input: runnable_with_memory.invoke(input,
                                                  config={"configurable": {"session_id": session_id}}
                                                  )
                                                  )
    return memory_by_session

In [37]:
SESSION_ID = "240518"

classify_message_with_memory = add_memory(classify_message_chain, SESSION_ID)
classify_message_with_memory

RunnableLambda(lambda input: runnable_with_memory.invoke(input, config={'configurable': {'session_id': session_id}}))

In [38]:
classify_message_with_memory_chain = RunnablePassthrough.assign(msg_type=classify_message_with_memory)
classify_message_with_memory_chain

RunnableAssign(mapper={
  msg_type: RunnableLambda(lambda input: runnable_with_memory.invoke(input, config={'configurable': {'session_id': session_id}}))
})

In [39]:
response = classify_message_with_memory_chain.invoke(
                {"user_id":3, 
                 "input": "주문할게요", 
                 "order_id": None},
            )
response

Parent run 3f144a0b-5633-4de0-8705-a4b105f2cbac not found for run 2d9668b9-03d2-428e-8cee-2317f61d0023. Treating as a root run.


{'user_id': 3,
 'input': '주문할게요',
 'order_id': None,
 'msg_type': AIMessage(content="입력해주신 메시지는 '요청' 유형에 해당합니다. 주문 요청을 하시려는 것으로 이해했습니다. 부가적인 정보가 필요하다면 언제든지 물어봐주세요.", response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 121, 'total_tokens': 187}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-80cac884-8b8b-41a2-942d-940bca470ab1-0')}